爬取政策链接
---
第一步（cell1：爬取政策链接）
在```url = "https://www.gov.cn/search/zhengce/?q=%E6%96%B0%E8%83%BD%E6%BA%90"```中爬取
其中```q = xxx```为关键词，直接搜索复制url即可

In [1]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# 配置 Edge 无头模式
edge_options = Options()
edge_options.add_argument("--headless")

# 实例化 Edge 浏览器
driver = webdriver.Edge(options=edge_options)

url = "https://sousuo.www.gov.cn/sousuo/search.shtml?code=17da70961a7&searchWord=%E6%99%BA%E8%83%BD%20%E4%BA%A4%E9%80%9A&dataTypeId=14&sign=c6db081f-a631-4440-f051-32774416dae9"
driver.get(url)

all_links = []

while True:
    try:
        # 等待页面中至少出现一个 a 标签
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "a")))
    except Exception as e:
        print("页面加载失败：", e)
        break

    # 获取当前页面渲染后的 HTML，并用 BS4 解析
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # 查找所有 class 为 "title log-anchor" 的 a 标签
    a_tags = soup.find_all("a", class_="title log-anchor")
    for a_tag in a_tags:
        href = a_tag.get("href")
        if href:
            all_links.append(href)
    
    try:
        # 定位下一页按钮（通过 class="next"）
        next_button = driver.find_element(By.CSS_SELECTOR, ".oPager a.next")
        # 此处可以判断下一页按钮是否禁用（例如检查是否有 disable/lose 标识）
        if "lose" in next_button.get_attribute("class"):
            print("翻页结束：下一页按钮不可点击")
            break

        next_button.click()
        # 等待页面更新
        time.sleep(3)
    except Exception as e:
        print("翻页结束或下一页按钮不可点击：", e)
        break

driver.quit()

with open("links.txt", "w", encoding="utf-8") as f:
    for link in all_links:
        f.write(link + "\n")

print("链接保存完成，共抓取到", len(all_links), "个链接")

翻页结束：下一页按钮不可点击
链接保存完成，共抓取到 11 个链接


In [ ]:
# reverse_urls.py

# 输入和输出文件路径（根据实际情况修改）
input_file = 'links.txt'    # 原始文件
output_file = 'urls_reversed.txt'  # 输出文件

# 读取原始文件内容
with open(input_file, 'r', encoding='utf-8') as f:
    # 读取所有行并去除空行
    lines = [line.strip() for line in f if line.strip()]

# 反转列表顺序
lines.reverse()

# 将倒序后的内容写入新文件
with open(output_file, 'w', encoding='utf-8') as f:
    for line in lines:
        f.write(f"{line}\n")

print(f"处理完成！已保存倒序后的文件至: {output_file}")

In [5]:
import os
import re
import time
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from bs4 import BeautifulSoup

# 从文件中读取所有链接
with open("urls_reversed.txt", "r", encoding="utf-8") as f:
    all_links = [line.strip() for line in f if line.strip()]

# 配置 Edge 的无头模式
edge_options = Options()
edge_options.add_argument("--headless")
driver = webdriver.Edge(options=edge_options)

# 创建保存txt文件的目录
output_dir = "../data/policy_spider/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 定义成功保存的文件计数器
saved_count = 0

for idx, link in enumerate(all_links):
    driver.get(link)
    # 预留等待时间，确保动态加载完成
    time.sleep(3)
    page_html = driver.page_source
    soup = BeautifulSoup(page_html, "html.parser")
    
    title = ""
    content = ""
    year = ""
    
    # 动态遍历标题的候选选择器
    title_candidates = [
        "div.mhide.pctoubukuang1 td[colspan='3']",
        "div.article.oneColumn h1#ti"
    ]
    for sel in title_candidates:
        tag = soup.select_one(sel)
        if tag:
            title = tag.get_text(strip=True)
            if title:
                break

    # 动态遍历日期（年份）的候选逻辑：
    # ① 尝试从包含 “成文日期” 的 td 后面获取
    date_found = False
    for td in soup.select("div.mhide.pctoubukuang1 td.lf15"):
        if "成文日期" in td.get_text():
            next_td = td.find_next_sibling("td")
            if next_td:
                time_text = next_td.get_text(strip=True)
                m = re.search(r'(\d{4})', time_text)
                if m:
                    year = m.group(1)
                    date_found = True
                    break
    # ② 尝试通用日期区域
    if not date_found:
        date_tag = soup.select_one("div.pages-date")
        if date_tag:
            time_text = date_tag.get_text(strip=True)
            m = re.search(r'(\d{4})', time_text)
            if m:
                year = m.group(1)
    
    # 动态遍历正文的候选选择器
    # 每个候选为 (selector, 提取方式)
    # 提取方式： "full" 为直接全文提取； "indent" 为仅提取 p 标签中包含 "text-indent: 2em" 的文本
    content_candidates = [
        ("table.border-table.noneBorder.pages_content div.b12c.pages_content#UCAP-CONTENT", "full"),
        ("div.trs_editor_view.TRS_UEDITOR.trs_paper_default.trs_web", "full"),
        ("div.view.TRS_UEDITOR.trs_paper_default.trs_web", "indent"),
        ("div.trs_editor_view.TRS_UEDITOR.trs_paper_default", "full"),
        ("div.pages_content.mhide", "full"),
        ("div.pages_content", "indent")  # 新增选择器，针对 pages_content 的 div
    ]
    for sel, method in content_candidates:
        tag = soup.select_one(sel)
        if tag:
            if method == "full":
                content = tag.get_text(separator="\n", strip=True)
            elif method == "indent":
                lines = []
                for p in tag.find_all("p"):
                    style = p.get("style", "")
                    if "text-indent: 2em" in style:
                        lines.append(p.get_text(strip=True))
                content = "\n".join(lines)
            # 只要有内容就认为匹配成功
            if content:
                break

    if not title or not content:
        print(f"无法获取有效内容：{link}")
        continue

    # 构造安全的文件名，格式为 "年份_标题.txt"（若未提取到年份，则仅用标题）
    safe_title = re.sub(r"[\\/*?\"<>|]", "", title)
    if year:
        file_name = f"{year}_{safe_title}.txt"
    else:
        file_name = f"{safe_title}.txt"
    file_path = os.path.join(output_dir, file_name)
    
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(title + "\n\n")
        f.write(content)
    
    saved_count += 1
    print(f"{file_path} 保存成功. 这是成功保存的第 {saved_count} 个文件.")

driver.quit()

../data/policy_spider/2016_实现旅客便捷出行、货物高效运输——国家发改委基础产业司副司长解读智能交通发展实施方案.txt 保存成功. 这是成功保存的第 1 个文件.
无法获取有效内容：http://www.gov.cn/zhengce/zhengceku/2018-12/31/content_5440057.htm
无法获取有效内容：http://www.gov.cn/zhengce/zhengceku/2018-12/31/content_5442955.htm
无法获取有效内容：http://www.gov.cn/zhengce/zhengceku/2019-10/15/content_5440071.htm
无法获取有效内容：http://www.gov.cn/zhengce/zhengceku/2019-11/19/content_5456289.htm
无法获取有效内容：http://www.gov.cn/zhengce/zhengceku/2019-11/21/content_5454148.htm
无法获取有效内容：http://www.gov.cn/zhengce/zhengceku/2020-12/28/content_5574286.htm
无法获取有效内容：http://www.gov.cn/zhengce/zhengceku/2021-03/20/content_5594095.htm
无法获取有效内容：http://www.gov.cn/zhengce/zhengceku/2021-11/10/content_5650034.htm
无法获取有效内容：http://www.gov.cn/zhengce/zhengceku/2022-09/25/content_5711801.htm
无法获取有效内容：https://www.gov.cn/zhengce/zhengceku/202404/content_6947020.htm


In [5]:
import os
import pandas as pd

# 定义 CSV 文件所在的文件夹路径
csv_folder = '../data/weibo_raw'
# 定义输出的 TXT 文件的文件名
output_txt = 'weibo_content.txt'

try:
    with open(output_txt, 'w', encoding='utf-8') as txt_file:
        # 遍历文件夹中的所有文件
        for root, dirs, files in os.walk(csv_folder):
            for file in files:
                if file.endswith('.csv'):
                    csv_file = os.path.join(root, file)
                    try:
                        # 读取 CSV 文件
                        df = pd.read_csv(csv_file)
                        # 检查是否存在“微博正文”列
                        if '微博正文' in df.columns:
                            # 获取“微博正文”列的数据
                            weibo_content = df['微博正文']
                            for content in weibo_content:
                                if pd.notna(content):
                                    # 将内容写入 TXT 文件
                                    txt_file.write(str(content) + '\n')
                        else:
                            print(f"文件 {csv_file} 中不存在 '微博正文' 列。")
                    except FileNotFoundError:
                        print(f"错误: 文件 {csv_file} 未找到。")
                    except Exception as e:
                        print(f"错误: 读取文件 {csv_file} 时发生未知错误: {e}")
except Exception as e:
    print(f"错误: 写入文件 {output_txt} 时发生未知错误: {e}")
    